In [3]:
import copy
import argparse
import pandas as pd
import random

In [4]:
df = pd.read_excel('5.2 Assignment - Tabu Search - Picker Routing Problem - Template.xlsx','Distances', index_col = None,na_values=['NA'], usecols = "AA:AN ", header = None)

In [5]:
df = df.drop([0,1,2], axis = 0)

In [6]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df= df.drop([17,18], axis = 0)

In [7]:
df

3,"i,j",0.0,5.0,16.0,24.0,43.0,50.0,52.0,56.0,59.0,68.0,89.0,96.0,100.0
4,0,0.0,7.0,8.0,14.0,21.0,24.0,11.0,11.0,14.0,17.0,26.0,27.0,31.0
5,5,7.0,0.0,11.0,11.0,20.0,19.0,4.0,4.0,7.0,10.0,19.0,20.0,24.0
6,16,8.0,11.0,0.0,8.0,15.0,18.0,13.0,11.0,14.0,9.0,18.0,19.0,23.0
7,24,14.0,11.0,8.0,0.0,13.0,12.0,13.0,11.0,14.0,9.0,12.0,13.0,17.0
8,43,21.0,20.0,15.0,13.0,0.0,3.0,22.0,20.0,23.0,18.0,11.0,6.0,10.0
9,50,24.0,19.0,18.0,12.0,3.0,0.0,21.0,19.0,22.0,17.0,10.0,3.0,7.0
10,52,11.0,4.0,13.0,13.0,22.0,21.0,0.0,2.0,3.0,10.0,19.0,20.0,22.0
11,56,11.0,4.0,11.0,11.0,20.0,19.0,2.0,0.0,3.0,8.0,17.0,18.0,22.0
12,59,14.0,7.0,14.0,14.0,23.0,22.0,3.0,3.0,0.0,9.0,16.0,21.0,19.0
13,68,17.0,10.0,9.0,9.0,18.0,17.0,10.0,8.0,9.0,0.0,13.0,16.0,16.0


In [309]:
def generate_neighbours(df):
    dict_of_neighbours = {}
    for idx, row in df.iterrows():
        ls = []
        for j in df.columns[1:]:
            if row['i,j'] != int(j):
                ls.append([int(j), df.at[idx,j]])    
        dict_of_neighbours[row['i,j']] = ls
 
    return dict_of_neighbours

dict_of_neighbours = generate_neighbours(df)

In [428]:
def generate_first_solution(df, dict_of_neighbours):
    first_solution = list(df['i,j'].values[1:])
    random.shuffle(first_solution)
    first_solution.insert(0, 0)
    first_solution.append(0)
    
    distance = 0
    for i in first_solution:
        next_point = first_solution[first_solution.index(16) +1]
        for j in dict_of_neighbours[i]:
            if j[0]== next_point:
                distance = distance + int(j[1])
    
    return first_solution, distance


In [429]:
def neighbor_structure(solution):
    neighbor_table = []
    for i in range(0,len(solution[1:-1])):
        for j in range(i+1, len(solution[1:-1])):
            neighbor_table.append([solution[1:-1][i],solution[1:-1][j], 0])
    return neighbor_table

In [430]:
def swap(i,j, solution):
    neighbor = solution.copy()
    pos_i = solution.index(i)
    pos_j = solution.index(j)
    temp=neighbor[pos_i]
    neighbor[pos_i]=neighbor[pos_j]
    neighbor[pos_j]=temp
    return neighbor

In [431]:
def cal_cost(solution):
    distance = 0
    for i in solution:
        next_point = solution[solution.index(i) +1]
        for j in dict_of_neighbours[i]:
            if j[0]== next_point:
                distance = distance + int(j[1])
    return distance

In [432]:
def search_in_table(table, point):
    for idx, row in table.iterrows():
        if set([row['i'],row['j']]) == set(point):
            table.iloc[idx]['tabu_list'] =  1     
    return table

In [435]:
def update_table(table):
    for idx, row in table.iterrows():
        if row['tabu_list'] !=0:
            table.iloc[idx]['tabu_list'] +=1
    return table

In [436]:
def tabu_search(neighbor_table,
                dict_of_neighbours,
                iterations,
                number_of_neighbors,
                size,
                first_solution,
                distance_of_first_solution ):
    
    solution = first_solution
    best_solution = first_solution
    best_cost = distance_of_first_solution
    count = 0
    tabu_list = {}
    
    structure = pd.DataFrame (neighbor_table, columns = ['i', 'j','tabu_list'])
    #print(structure)
    while count <= iterations:
        # randomly take the number of neighbors
        temp = structure[structure['tabu_list']==0].sample(n= number_of_neighbors)
        
        solution_space = []
        points = []
        for idx, row in temp.iterrows():
            a = swap(row['i'], row['j'], solution)
            solution_space.append(a)
            points.append([row['i'], row['j']])

        # check logic 
        cost = [cal_cost(sol) for sol in solution_space]
        
        # get the best swap and solution
        point = points[cost.index(min(cost))]
        solution = solution_space[cost.index(min(cost))]
        if int(min(cost)) < best_cost:
            best_cost = int(min(cost))
            best_solution = solution
        
        structure = search_in_table(structure, point) #update table
        structure = update_table(structure)
        if len(structure['tabu_list']) >= size:
            structure['tabu_list'] = structure['tabu_list'].replace(size, 0)
        
        count +=1
    return best_solution, best_cost     
        

In [451]:
iterations = 5
size =1
number_of_neighbors = 3
first_solution, distance_of_first_solution = generate_first_solution(df, dict_of_neighbours)
neighbor_table= neighbor_structure(first_solution)
for i in range(10):
    best_sol, best_cos = tabu_search(neighbor_table,
                    dict_of_neighbours,
                    iterations,
                    number_of_neighbors,
                    size,
                    first_solution,
                    distance_of_first_solution)
    
    print(f"Best solution: {best_sol}, with total distance: {best_cos}.")
    i +=1

Best solution: [0, 24, 16, 52, 89, 96, 43, 56, 5, 50, 59, 68, 100, 0], with total distance: 134.
Best solution: [0, 24, 16, 52, 89, 96, 59, 56, 68, 50, 5, 43, 100, 0], with total distance: 140.
Best solution: [0, 24, 16, 59, 68, 96, 5, 89, 100, 43, 50, 52, 56, 0], with total distance: 104.
Best solution: [0, 24, 16, 52, 5, 50, 59, 56, 68, 43, 96, 100, 89, 0], with total distance: 120.
Best solution: [0, 24, 16, 68, 59, 43, 89, 96, 100, 50, 56, 5, 52, 0], with total distance: 136.
Best solution: [0, 24, 16, 59, 68, 96, 52, 56, 89, 50, 43, 5, 100, 0], with total distance: 166.
Best solution: [0, 24, 16, 56, 52, 96, 50, 59, 68, 89, 43, 5, 100, 0], with total distance: 128.
Best solution: [0, 24, 16, 52, 89, 96, 50, 56, 59, 68, 5, 43, 100, 0], with total distance: 98.
Best solution: [0, 52, 5, 43, 96, 50, 59, 56, 89, 68, 24, 16, 100, 0], with total distance: 121.
Best solution: [0, 24, 16, 68, 89, 56, 59, 96, 43, 50, 52, 5, 100, 0], with total distance: 122.


In [452]:
iterations = 10
size =3
number_of_neighbors = 6
first_solution, distance_of_first_solution = generate_first_solution(df, dict_of_neighbours)
neighbor_table= neighbor_structure(first_solution)
for i in range(10):
    best_sol, best_cos = tabu_search(neighbor_table,
                    dict_of_neighbours,
                    iterations,
                    number_of_neighbors,
                    size,
                    first_solution,
                    distance_of_first_solution)
    
    print(f"Best solution: {best_sol}, with total distance: {best_cos}.")
    i = i +1

Best solution: [0, 5, 100, 43, 89, 68, 96, 50, 16, 56, 52, 59, 24, 0], with total distance: 91.
Best solution: [0, 59, 5, 96, 24, 43, 100, 50, 52, 56, 89, 68, 16, 0], with total distance: 112.
Best solution: [0, 52, 5, 100, 89, 96, 43, 50, 59, 56, 24, 16, 68, 0], with total distance: 109.
Best solution: [0, 24, 5, 96, 43, 100, 50, 52, 56, 89, 16, 59, 68, 0], with total distance: 100.
Best solution: [0, 5, 96, 50, 52, 59, 56, 100, 43, 89, 68, 24, 16, 0], with total distance: 71.
Best solution: [0, 16, 5, 50, 56, 100, 43, 52, 59, 68, 96, 89, 24, 0], with total distance: 108.
Best solution: [0, 59, 5, 50, 68, 96, 43, 100, 89, 56, 52, 24, 16, 0], with total distance: 116.
Best solution: [0, 56, 5, 50, 52, 96, 43, 100, 89, 24, 16, 59, 68, 0], with total distance: 97.
Best solution: [0, 5, 96, 43, 68, 89, 24, 16, 52, 56, 59, 100, 50, 0], with total distance: 91.
Best solution: [0, 56, 59, 52, 5, 96, 68, 100, 50, 43, 89, 24, 16, 0], with total distance: 121.


In [449]:
iterations = 20
size =6
number_of_neighbors = 12
first_solution, distance_of_first_solution = generate_first_solution(df, dict_of_neighbours)
neighbor_table= neighbor_structure(first_solution)
for i in range(10):
    best_sol, best_cos = tabu_search(neighbor_table,
                    dict_of_neighbours,
                    iterations,
                    number_of_neighbors,
                    size,
                    first_solution,
                    distance_of_first_solution)
    
    print(f"Best solution: {best_sol}, with total distance: {best_cos}.")
    i = i +1

Best solution: [0, 16, 5, 43, 100, 89, 96, 50, 59, 68, 24, 56, 52, 0], with total distance: 70.
Best solution: [0, 5, 100, 43, 89, 96, 50, 52, 56, 59, 68, 24, 16, 0], with total distance: 41.
Best solution: [0, 52, 5, 50, 56, 59, 68, 89, 100, 43, 96, 24, 16, 0], with total distance: 73.
Best solution: [0, 5, 89, 100, 43, 96, 50, 68, 24, 16, 52, 56, 59, 0], with total distance: 65.
Best solution: [0, 5, 50, 56, 59, 96, 100, 43, 89, 68, 24, 16, 52, 0], with total distance: 77.
Best solution: [0, 5, 100, 43, 89, 96, 50, 56, 59, 68, 24, 16, 52, 0], with total distance: 51.
Best solution: [0, 59, 56, 68, 24, 5, 96, 43, 89, 100, 50, 52, 16, 0], with total distance: 94.
Best solution: [0, 16, 59, 56, 52, 5, 100, 43, 89, 96, 50, 68, 24, 0], with total distance: 76.
Best solution: [0, 16, 5, 100, 43, 96, 50, 59, 89, 68, 24, 56, 52, 0], with total distance: 78.
Best solution: [0, 5, 89, 96, 43, 100, 50, 52, 56, 59, 68, 16, 24, 0], with total distance: 65.
